In [ ]:
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune, air
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.ppo import PPO, PPOConfig

from bmstestbedc2f2.envs import _TODO_MultiAgentBuildingEnv
from bmstestbedc2f2.utils import resolve_path


pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    resample_probability=0.25,
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "batch_mode": ["complete_episodes"],
        "train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
    },
)

tuner = tune.Tuner(
    PPO,
    param_space=(
        _TODO_MultiAgentBuildingEnv.get_algo_config(
            PPOConfig()
            .api_stack(
                enable_rl_module_and_learner=False,
                enable_env_runner_and_connector_v2=False,
            )
            .rollouts(
                sample_timeout_s=60,
                num_env_runners=4, 
                rollout_fragment_length='auto',
                # rollout_fragment_length=200,
            )
            .resources(num_gpus=1.),
            env_config=dict(
                bms_system='energyplus',
            ),
        )
    ),
    tune_config=tune.TuneConfig(
        #reuse_actors=True,
        scheduler=pbt,
        num_samples=4,
        metric="env_runners/episode_reward_mean",
        mode="max",
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True
        ),
        verbose=2,
    )
)
results = tuner.fit()

/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-01 09:36:59,247	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-01 09:36:59,527	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
E0000 00:00:1738402619.809243   70470 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738402619.812593   70470 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 09:37:05,846	WARNING de

(pid=72114) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=72114) E0000 00:00:1738402627.749482   72114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=72114) E0000 00:00:1738402627.752689   72114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=72244) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=72244) E0000 00:00:1738402633.861790   72244 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=72244) E0000 00:00:1738402633.865202   72244 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=72245) WARNING: All log messages before abs

Trial name
PPO__TODO_MultiAgentBuildingEnv_1a129_00000
PPO__TODO_MultiAgentBuildingEnv_1a129_00001


(PPO pid=72114) 2025-02-01 09:37:23,043	ERROR actor_manager.py:804 -- Ray error (ray::RolloutWorker.apply() (pid=72244, ip=192.168.200.249, actor_id=37686aa55c9ec4e585702b6901000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f2ec14071d0>)
(PPO pid=72114)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=72114)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=72114)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/utils/actor_manager.py", line 194, in apply
(PPO pid=72114)     raise e
(PPO pid=72114)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/utils/actor_manager.py", line 183, in apply
(PPO pid=72114)     return func(self, *args, **kwargs)
(PPO pid=72114)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=72114)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/execution/rollout_ops.py", line 102, in <lambda>
(PPO pid=72114)     (lambda w:

(RolloutWorker pid=72965) [2025-02-01 09:37:50,920 C 72965 72965] task_receiver.cc:185:  Check failed: _s.ok() Bad status: IOError: Broken pipe
(RolloutWorker pid=72965) *** StackTrace Information ***
(RolloutWorker pid=72965) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/_raylet.so(+0x1141c3a) [0x7f573b741c3a] ray::operator<<()
(RolloutWorker pid=72965) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/_raylet.so(_ZN3ray6RayLogD1Ev+0x4d1) [0x7f573b7442c1] ray::RayLog::~RayLog()
(RolloutWorker pid=72965) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/_raylet.so(+0x8e3576) [0x7f573aee3576] ray::core::TaskReceiver::HandleTask()::{lambda()#1}::operator()()
(RolloutWorker pid=72965) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/_raylet.so(+0x8e3cfa) [0x7f573aee3cfa] std::_Function_handler<>::_M_invoke()
(RolloutWorker pid=72965) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packa

In [2]:
best_result = results.get_best_result()
best_result.checkpoint.to_directory(resolve_path('algo_checkpoint'))
best_result

/usr/lib/python3.11/ast.py:50: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2025-01-22_16-38-22_357299_58441/artifacts/2025-01-22_16-38-23/PPO_2025-01-22_16-38-22/driver_artifacts/PPO_MultiAgentBuildingEnv_4bfe2_00000_0_2025-01-22_16-38-23/result.json' mode='at' encoding='UTF-8'>
  return compile(source, filename, mode, flags,
/usr/lib/python3.11/ast.py:50: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2025-01-22_16-38-22_357299_58441/artifacts/2025-01-22_16-38-23/PPO_2025-01-22_16-38-22/driver_artifacts/PPO_MultiAgentBuildingEnv_4bfe2_00000_0_2025-01-22_16-38-23/progress.csv' mode='at' encoding='UTF-8'>
  return compile(source, filename, mode, flags,


AttributeError: 'NoneType' object has no attribute 'to_directory'